In [0]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [0]:
#https://medium.com/@14prakash/transfer-learning-using-keras-d804b2e04ef8

from keras import applications
import cv2
import re
import os
import random
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,Conv1D,Conv2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import numpy as np
from keras.utils import to_categorical
from sklearn.utils import shuffle
img_width=256
img_height=256

In [0]:


def test_train_dev_split(input_data,output_data,train=0.7,dev=0.2,test=0.1):
    #make seed for exact results everything
    #random.sort(dataset)
    #np.random.seed(2)
    #np.random.shuffle(input_data)
    #np.random.shuffle(output_data)
    input_data, output_data = shuffle(input_data, output_data, random_state=0)
    split1=int(train*len(input_data))
    split2=int((train+dev)*len(input_data))
    train_input=input_data[:split1]
    dev_input=input_data[split1:split2]
    test_input=input_data[split2:]
    
    
    train_output=output_data[:split1]
    dev_output=output_data[split1:split2]
    test_output=output_data[split2:]
    
    return np.array(train_input),np.array(train_output),np.array(dev_input),np.array(dev_output),np.array(test_input),np.array(test_output)

In [0]:

oneHot2Name={}

def load_small_img_dataset(path=os.getcwd()):
    data=[]
    labels=[]
    for image in os.listdir(path):
        if image.endswith(('.jpg','.jpeg','.png','bmp'),0,len(image)):
            label=re.findall('^(.*)_.*',image)[0]
            labels.append(label)
            pixels=cv2.imread(os.path.join(path,image))
            pixels=cv2.resize(pixels,(256,256),interpolation=cv2.INTER_CUBIC)
            
            #first make all images of same size using crop
            data.append(pixels)
            
    return data,labels

x,y=load_small_img_dataset('D:\projects\ATM public safety\Third review\Face Recognition\Face_Dataset')





persons=set(y)
num_persons=len(set(y))
categorical_mapping={}

#convert into one hot encoding
for i,name in enumerate(persons):
    #print(i,name)
    categorical_mapping[name]=i

output_d=[]

for i_ in y:
    i_=categorical_mapping[i_]
    output_d.append(i_)
output_d=to_categorical(output_d)



train_input,train_output,dev_input,dev_output,test_input,test_output=test_train_dev_split(x,output_d)
for i in categorical_mapping.keys():
    oneHot2Name[categorical_mapping[i]]=i




oneHot2Name
cv2.imshow(oneHot2Name[np.argmax(train_output[13])],train_input[13])
cv2.waitKey()
cv2.destroyAllWindows()


# to convert our data type to float32 and normalize our database
train_input=train_input.astype('float32')
dev_input=dev_input.astype('float32')
test_input=test_input.astype('float32')
print(train_input.shape)
print(test_input.shape)


# Z-scoring or Gaussian Normalization
train_input=train_input - np.mean(train_input) / train_input.std()
dev_input=dev_input - np.mean(dev_input) / dev_input.std()
test_input=test_input - np.mean(test_input) / test_input.std()
categorical_mapping


train_input=train_input/255.0
dev_input=dev_input/255.0
test_input=test_input/255.0

#Saving into numpy arrays
np.save('train_in.npy',train_input)
np.save('train_out.npy',train_output)
np.save('dev_in.npy',dev_input)
np.save('dev_out.npy',dev_output)
np.save('test_in.npy',test_input)
np.save('test_out.npy',test_output)
np.save('num_persons.npy',num_persons)
np.save('categorical_mapping.npy',categorical_mapping)



In [0]:
# Load data from saved numpy arrays


train_input=np.load('train_in.npy')
train_output=np.load('train_out.npy')
dev_input=np.load('dev_in.npy')
dev_output=np.load('dev_out.npy')
test_input=np.load('test_in.npy')
test_output=np.load('test_out.npy')
num_persons=np.load('num_persons.npy')
categorical_mapping=np.load('categorical_mapping.npy').item()



In [0]:
'''# for olittivia face dataset 95% accuracy

x=np.load('olivetti_faces.npy')
y=np.load('olivetti_faces_target.npy')


y=y.reshape((len(y),1))
y=to_categorical(y)
num_persons=40

train_input,train_output,dev_input,dev_output,test_input,test_output= test_train_dev_split(x,y)
y[0]


'''

In [4]:
'''model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
for layer in model.layers[:19]:
    layer.trainable = False'''
categorical_mapping

{'Das': 0,
 'Faisal': 4,
 'Inder': 7,
 'Mummy': 5,
 'Nikhil': 8,
 'NikhilAgarwal': 9,
 'Papa': 6,
 'Ravi': 1,
 'Surya': 2,
 'Vandana': 3}

In [7]:
#Adding custom Layers 
model=Sequential()
model.add(Conv2D(8,5,input_shape=(256,256,3),activation='relu'))
#model.add(Conv2D(9,5,activation='relu'))
model.add(Flatten())
'''model.add(Dense(64, activation="relu"))
model.add(Dropout(0.3))'''
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.4))
'''model.add(Dense(16, activation="relu"))
model.add(Dropout(0.2))'''
model.add(Dense(num_persons, activation="softmax"))

'''x = model.output
x = Flatten()(x)
x = Dense(12, activation="relu")(x)
x = Dropout(0.3)(x)
predictions = Dense(num_persons, activation="sigmoid")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

sg = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True
'''
model_final=model
sg = optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
ad=optimizers.adam(0.001)
# compile the model 
model_final.compile( optimizer=sg, loss = "categorical_crossentropy", metrics=["accuracy"])


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=500, verbose=1, mode='auto')

'''
# Train the model 
model_final.fit_generator(
train_generator,
samples_per_epoch = nb_train_samples,
epochs = epochs,
validation_data = validation_generator,
nb_val_samples = nb_validation_samples,
callbacks = [checkpoint, early])
'''
TensorBoard("logs/run_a")
tensboard=TensorBoard("logs/run_a")
model_final.fit(train_input,train_output,batch_size=10, epochs=1000,validation_data=(dev_input,dev_output),callbacks = [checkpoint])

'''
#OR
model_final.load_weights('FaceRecognition_clean69_weights.h5')

'''

Instructions for updating:
Use tf.cast instead.
Train on 166 samples, validate on 48 samples
Epoch 1/1000
166/166 [==============================] - 6s 39ms/step - loss: 2.3386 - acc: 0.1506 - val_loss: 2.1124 - val_acc: 0.2500

Epoch 00001: val_acc improved from -inf to 0.25000, saving model to vgg16_1.h5
Epoch 2/1000
166/166 [==============================] - 1s 4ms/step - loss: 2.1253 - acc: 0.1807 - val_loss: 2.0532 - val_acc: 0.3750

Epoch 00002: val_acc improved from 0.25000 to 0.37500, saving model to vgg16_1.h5
Epoch 3/1000
166/166 [==============================] - 1s 4ms/step - loss: 2.0705 - acc: 0.2289 - val_loss: 2.0762 - val_acc: 0.2083

Epoch 00003: val_acc did not improve from 0.37500
Epoch 4/1000
166/166 [==============================] - 1s 4ms/step - loss: 2.0464 - acc: 0.2048 - val_loss: 1.9562 - val_acc: 0.3333

Epoch 00004: val_acc did not improve from 0.37500
Epoch 5/1000
166/166 [==============================] - 1s 4ms/step - loss: 2.0294 - acc: 0.2229 - val_lo

"\n#OR\nmodel_final.load_weights('FaceRecognition_clean69_weights.h5')\n\n"

In [9]:
score=model_final.evaluate(test_input,test_output)
#print(score)
print("[INFO] Loss:{0}   Accuracy:{1}".format(score[0],score[1]))


24/24 [==============================] - 0s 6ms/step
[INFO] Loss:1.9805556535720825   Accuracy:0.5416666865348816


In [16]:
#to improve Model try
'''
Class balancing
eigen faces or other pre-processing


'''

[[0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [0]:
#Save the model
model.save('FaceRecognition_custom_data.h5')
jsonmodel=model.to_json()
model.save_weights('FaceRecognition_custom_data_weights.h5')


In [12]:
test_predictions=model_final.predict(test_input)
print(test_predictions)


[[1.16110868e-06 7.08472550e-01 1.20596610e-06 9.83669906e-07
  1.89155188e-07 9.56792610e-06 1.69290081e-01 4.62690605e-06
  6.11053622e-07 1.22219019e-01]
 [1.94253703e-03 1.96399982e-03 3.69035751e-01 1.72315910e-01
  5.80144871e-04 2.85800785e-01 8.19312874e-04 6.24623522e-03
  2.06963811e-03 1.59225687e-01]
 [8.85455353e-09 9.69811342e-09 6.96188092e-01 7.07605779e-02
  1.79968623e-10 1.81048974e-01 9.83232273e-10 3.08643820e-07
  2.54409311e-08 5.20021059e-02]
 [2.42665652e-02 4.07973528e-01 1.70670636e-02 1.52358254e-02
  1.61820259e-02 3.17047499e-02 2.46889621e-01 3.57246473e-02
  1.57651752e-02 1.89190805e-01]
 [2.34424139e-23 3.16688021e-23 9.82807040e-01 1.83368661e-03
  4.14861873e-28 1.46199884e-02 7.86789899e-26 4.22747738e-19
  9.07815475e-22 7.39236129e-04]
 [1.23135790e-01 1.20179273e-01 7.79173300e-02 7.12798089e-02
  1.20321959e-01 9.76483896e-02 9.35635716e-02 1.38389662e-01
  8.46986249e-02 7.28656352e-02]
 [8.83094515e-17 1.08596374e-16 9.33973253e-01 1.04488675e

In [13]:
oneHot2Name={}

for i in categorical_mapping.keys():
    oneHot2Name[categorical_mapping[i]]=i
oneHot2Name

{0: 'Das',
 1: 'Ravi',
 2: 'Surya',
 3: 'Vandana',
 4: 'Faisal',
 5: 'Mummy',
 6: 'Papa',
 7: 'Inder',
 8: 'Nikhil',
 9: 'NikhilAgarwal'}

In [14]:
c=0
for i in test_predictions:
    #cv2.imshow(str(oneHot2Name[np.argmax(i)])+str(c)+'.jpg',test_input[c])
    cv2.imwrite('output_'+str(oneHot2Name[np.argmax(i)])+str(c)+'.jpg',test_input[c]*255)
    c+=1
    
'''cv2.waitKey(0)
cv2.destroyAllWindows()'''

'cv2.waitKey(0)\ncv2.destroyAllWindows()'

/bin/bash: nbconvert: command not found


In [0]:
1/14

0.07142857142857142